In [2]:
import requests
import pandas as pd

In [3]:
# Verify if API is ON

result_on = requests.get('http://localhost:5000/')
result_on.text

'{"message":"API available","status":"success"}\n'

In [4]:
#Load insurance dataset
df = pd.read_csv('insurance.csv')

In [5]:
# Analysing columns
df.head()

,age,sex,bmi,children,smoker,region,charges
0,19,female,27.900,0,yes,southwest,16884.92400
1,18,male,33.770,1,no,southeast,1725.55230
2,28,male,33.000,3,no,southeast,4449.46200
3,33,male,22.705,0,no,northwest,21984.47061
4,32,male,28.880,0,no,northwest,3866.85520


In [6]:
df.dtypes

age           int64
sex          object
bmi         float64
children      int64
smoker       object
region       object
charges     float64
dtype: object

In [7]:
# PREPARE Endpoint
df_json = df.to_json()
post_data = {'max_na': 0.2, 'max_var': 0.2, 'payload': df_json}
url = 'http://localhost:5000/prepare'

result_prepare = requests.post(url, data=post_data)
df_prep = pd.read_json(result_prepare.text)


In [8]:
df_prep.head()

,age,sex,bmi,children,smoker,region,charges
0,19,0,27.900,0,1,3,16884.92400
1,18,1,33.770,1,0,2,1725.55230
2,28,1,33.000,3,0,2,4449.46200
3,33,1,22.705,0,0,1,21984.47061
4,32,1,28.880,0,0,1,3866.85520


In [9]:
df_prep.dtypes

age           int64
sex           int64
bmi         float64
children      int64
smoker        int64
region        int64
charges     float64
dtype: object

In [10]:
# TRAIN_TEST Endpoint
url = 'http://localhost:5000/train_test'
ml_method = 'Regression'
train_columns = ''
target = 'charges'
df_json = df_prep.to_json()
post_data = {'train_columns': ','.join(train_columns), 'target_column': target, 'ml_method': ml_method, 'test_size': 0.2, 'payload': df_json}
result_train_test = requests.post(url, data=post_data)
print(result_train_test.text)

{"gradient":{"MAE":2468.856784533358,"MSE":19236349.643856116,"RMSE":4385.9263153701195},"linear":{"MAE":4024.4442885471926,"MSE":33518303.67739486,"RMSE":5789.499432368472},"randon_forest":{"MAE":2680.283741332556,"MSE":22282753.91316965,"RMSE":4720.461197083358}}



In [11]:
#TRAIN Endpoint
url = 'http://localhost:5000/train_model'
post_data = {'train_columns': ','.join(train_columns), 'target_column': target, 'ml_method': ml_method, 'model_name': 'gradient', 'payload': df_json}
result_train = requests.post(url, data=post_data)
print(result_train.text)

{"file_name":"models_registry/2021-12-29-17-42-44/gradient_regressor.pickle","file_name_param":"models_registry/2021-12-29-17-42-44/gradient_regressor.json"}



In [12]:
#PREDICT Endpoint
df_predict = df_prep.drop('charges', axis=1)
df_json = df_predict.to_json()
model_path = result_train.text.split(',')[0].split(':')[1].replace('"','').replace(' ','')
url = 'http://localhost:5000/predict'
post_data = {'model_path': model_path, 'payload': df_json}

result_predict = requests.post(url, data=post_data)
print(result_predict.text)

{"prediction":[18901.83209495813,3825.97363096915,7069.608277896786,5774.003690349495,4146.165127216481,3968.8316164354046,9081.802886661178,8114.112147602289,8374.8243898319,14437.666712661117,4126.249500146986,27695.84193937267,3185.5202897071813,12016.58435153292,38624.13821788517,3022.9121624570025,13192.169833992286,4121.8200731885345,11620.053083901135,38511.89881652495,15022.553018780547,5111.272411777457,2406.2484878092087,38692.97840252537,7629.663711963266,16070.431434563794,14571.995936513695,14096.20461450206,4821.888689960618,40010.64433496563,36457.51743486965,3016.21325730916,6125.305841870205,14199.730808069193,40318.21188435593,1547.0422882967594,17095.571092347924,2099.3213790214472,42407.27051237796,47333.58003324156,5029.581424911623,6501.239872725893,6733.190538406885,7285.82211333053,7893.250305785391,12959.799737195595,5953.225526386431,4258.142910126288,14484.40603902601,39178.11375122872,4237.325928862212,6870.693736991751,26104.14479107861,38332.977129760024,7